In [1]:
# 导入同花顺包和登录同花顺
import datetime as dt
import pandas as pd
from numpy import *
from iFinDPy import *
THS_iFinDLogin("gdzq6888", "199212")

D:\Anaconda\Lib\site-packages\iFinDPy.pth


0

In [126]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from calendar import monthrange
import numpy as np

In [13]:
# 定义给stock code分组的方程
def split(list_a, chunk_size):
    for i in range(0, len(list_a), chunk_size):
        yield list_a[i:i + chunk_size]

In [14]:
# 定义计算相关性平均值的方程
def find_upper_triangle(matrix):
    mask = np.triu(np.ones(matrix.shape), k=1)
    return matrix[mask.astype(bool)]

def solve(a):
    return find_upper_triangle(a)

# 提取数据

In [75]:
# 中证800
CSI800 = THS_DR('p03473','iv_date=20230203;iv_zsdm=000906.SH','p03473_f001:Y,p03473_f002:Y,p03473_f003:Y','format:dataframe').data

In [79]:
# 中证800回报数据
CSI800_return = THS_HQ('000906.SH','changeRatio,turnoverRatio,close','','2013-02-02','2023-02-02').data
CSI800_return['lag'] =CSI800_return.groupby(['thscode'])['close'].shift(1)
CSI800_return['return'] = (CSI800_return['close'] - CSI800_return['lag'])/CSI800_return['lag']
CSI800_return['month'] = CSI800_return['time'].str[:7]

# 多头alpha

## 个股截面离散度

In [58]:
# 输入日期格式为’年-月‘
def individual_stock_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index',month_end+';000906.SH','date:Y,thscode:Y').data.rename(columns={'THSCODE':'thscode'})
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return dispersion


In [59]:
individual_stock_dispersion('2023-01')

AttributeError: 'NoneType' object has no attribute 'rename'

In [65]:
THS_DP('index','2023-01-31'+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data

In [64]:
data

In [49]:
THS_HQ('430047.BJ','close','','2013-01-31','2023-02-01').data

,time,thscode,close
0,2020-07-27,430047.BJ,8.17
1,2020-07-28,430047.BJ,8.17
2,2020-07-29,430047.BJ,8.17
3,2020-07-30,430047.BJ,8.17
4,2020-07-31,430047.BJ,8.17
...,...,...,...
604,2023-01-19,430047.BJ,14.66
605,2023-01-20,430047.BJ,14.80
606,2023-01-30,430047.BJ,14.82
607,2023-01-31,430047.BJ,14.88


## 行业截面离散度

In [61]:
def industry_dispersion(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()

    # 计算日标准差
    std = data.groupby(['time'])['return'].std().reset_index()
    std = std.rename(columns={'return':'daily std'})
    std['month'] = std['time'].str[:7]

    # 得到离散度
    std_monthly = std.groupby('month')['daily std'].mean().reset_index()
    std_monthly = std_monthly.rename(columns={'daily std':'monthly std'})

    # 计算高/低离散度
    dispersion = list(std_monthly['monthly std'])[-1]/mean(list(std_monthly['monthly std']))

    return  dispersion


In [62]:
industry_dispersion('2023-01')

0.9181010129251563

## 个股相关性

In [50]:
# 输入日期格式为’年-月‘
def stock_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]
    # 获取中证800指数成分
    data = THS_DP('index_weight',month_end+';000906.SH','date:Y,thscode:Y,security_name:Y,weight:Y').data
    # 提取股票代码
    stock_code = list(data['thscode'])
    # 将股票代码分为4份
    stock_code2 = list(split(stock_code,200))
    #获得成分股每日价格
    data = pd.DataFrame()
    for i in stock_code2:
        a = THS_HQ(i,'close','',month_end_10years,month_end).data
        data=pd.concat([data,a])

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    # 计算相关性
    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation



In [51]:
stock_correlation('2023-01')

TypeError: 'NoneType' object is not subscriptable

## 行业相关性

In [52]:
def industry_correlation(time):
    date = datetime.strptime(time, '%Y-%m')
    days_in_month = monthrange(date.year, date.month)[1]
    month_end = date + timedelta(days=days_in_month - date.day)
    month_end_10years = date.replace(year=date.year-10)
    month_end = str(month_end)[:10]
    month_end_10years = str(month_end_10years)[:10]

    # 获取一级行业指数回报
    data=THS_HQ('CI005001.CI,CI005002.CI,CI005003.CI,CI005004.CI,CI005005.CI,CI005006.CI,CI005007.CI,CI005008.CI,CI005009.CI,CI005010.CI,CI005011.CI,CI005012.CI,CI005013.CI,CI005014.CI,CI005015.CI,CI005016.CI,CI005017.CI,CI005018.CI,CI005019.CI,CI005020.CI,CI005021.CI,CI005022.CI,CI005023.CI,CI005024.CI,CI005025.CI,CI005026.CI,CI005027.CI,CI005028.CI,CI005029.CI,CI005030.CI','close','',month_end_10years,month_end).data

    # 计算回报
    data['lag'] = data.groupby(['thscode'])['close'].shift(1)
    data['return'] = (data['close']-data['lag'])/data['lag']
    data = data.dropna()
    data['month'] = data['time'].str[:7]

    corr_list=list()
    for k in list(unique(data['month'])):
        df = pd.DataFrame()
        test = data[data['month']==k]
        for i in list(unique(test['thscode'])):
            a = test[test['thscode'] == i].reset_index()
            a = a[['return']]
            a = a.rename(columns={'return':i})
            df = pd.concat([df,a],axis=1)
        corr_matrix = df.corr().fillna(0).to_numpy()
        mean_correlation = mean(solve(corr_matrix))
        corr_list.append(mean_correlation)
    correlation_mean = mean(corr_list)

    correlation = corr_list[-1]/correlation_mean

    return correlation

In [53]:
industry_correlation('2023-01')

0.3673471098354658

## 市场流动性

In [171]:
def market_liquidity(start_date, end_date, data = CSI800_return):
    # 生成日流动性列
    data['日低流动性'] = abs(data['changeRatio']/data['turnoverRatio'])
    # 根据时间，从原数据表中筛选数据
    data_start_point = str(datetime.strptime(start_date, '%Y-%m-%d')-timedelta(days = 365*5))[:10]
    data = data[data['time']>=data_start_point]
    # 计算月流动性
    daily_liquidity = data.groupby('month')['日低流动性'].mean()
    # 计算两个日期之间有多少个月
    month_spread = relativedelta(datetime.strptime(end_date, '%Y-%m-%d'),datetime.strptime(start_date, '%Y-%m-%d')).months+1
    # 根据两个日期之间的月份，求每个月的流动性与它前5年的比值
    liquidity_list = list()
    for i in range(-1,-month_spread-1,-1):
        liquidity = daily_liquidity[i]/mean(daily_liquidity[i-60:i])
        liquidity_list.append(liquidity)
    liquidity_list.reverse()
    # 生成高/低流动性判断
    monthly_liquidity = pd.DataFrame(liquidity_list,index=daily_liquidity.index[-2:]).rename(columns= {0:'monthly_liquidity'})
    monthly_liquidity['high/low'] = np.where(monthly_liquidity['monthly_liquidity']>=1,'低流动性','高流动性')

    return monthly_liquidity

In [172]:
market_liquidity('2022-12-15','2023-01-21')

,monthly_liquidity,high/low
month,,
2023-01,0.732683,高流动性
2023-02,0.610809,高流动性
